In [1]:
from MesoPy import Meso
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pandas as pd

In [2]:
m = Meso(token='0191c61bf7914bd49b8bd7a98abb9469')

In [3]:
a = m.timeseries('202002262339', '202003111605', stid = 'AMBC1', units = 'height|m')['STATION'][0]['OBSERVATIONS']['air_temp_set_1']
b = [elem for elem in a if elem is not None]
np.nanmean(b)

1.7222852512155589

In [4]:
from glob import glob
import rasterio as rio

In [6]:
# glob('/scratch/zacharykeskinen/jpl/*/tiffs/*')
fp = '/bsuscratch/zacharykeskinen/data/uavsar/images/Lowman, CO/lowman_05208_21012-004_21015-009_0021d_s01_L090_01_int_grd/lowman_05208_21012-004_21015-009_0021d_s01_L090HV_01.cor.grd.tiff'
with rio.open(fp) as src:
    bounds  = src.bounds

In [29]:
from rasterio.windows import Window

In [30]:
with rio.open(fp) as src:
    meta = src.meta
    x_coord, y_coord = -115.23447, 44.30342
    box_side = 1
    a = src.read()
    print(a[~np.isnan(a)])
    rowcol = rio.transform.rowcol(meta['transform'], xs=x_coord, ys=y_coord)
    w = src.read(1, window=Window(rowcol[1], rowcol[0], box_side, box_side))
    # w = raster_box_extract(src, -115.23447, 44.30342)

[0.91771144 0.91771144 0.1554647  ... 0.23776883 0.28682545 0.28682545]


In [31]:
w

array([[0.5323299]], dtype=float32)

In [5]:
def raster_box_extract(img, x_coord, y_coord, box_side = 5):
    meta = img.meta

    # Use the transform in the metadata and your coordinates
    try:
        rowcol = rio.transform.rowcol(meta['transform'], xs=x_coord, ys=y_coord)
        w = img.read(1, window=Window(rowcol[1], rowcol[0], box_side, box_side))
        return w
    except:
        pass
        # print('station outside bbox')

In [10]:
w

In [52]:
stat_ls = []
stid_ls = []
res = {}
start_flight = '202002010000'
end_flight = '202003010000'

for stat in m.metadata(start = start_flight, end = end_flight, bbox = bounds)['STATION']:
    long = float(stat['LONGITUDE'])
    lat = float(stat['LATITUDE'])
    with rio.open(fp) as src:
        w = raster_box_extract(src, long, lat)
    if len(w[~np.isnan(w)]) > 0:
        if stat['NAME'].lower().replace(' ','') not in stat_ls:
            obs = m.timeseries(start_flight, end_flight, stid = stat['STID'], vars = ['snow_depth', 'snow_water_equiv'])
            if obs:
                obs = obs['STATION'][0]['OBSERVATIONS']
                d = {}
                dt = pd.to_datetime(obs['date_time'])
                if 'snow_depth_set_1' in obs.keys() and 'snow_water_equiv_set_1' in obs.keys():
                    d['datetime'] = dt
                    d['img_arr'] = w
                    d['snow_depth_set_1'] = obs['snow_depth_set_1']
                    d['swe_set_1'] = obs['snow_water_equiv_set_1']
                    for anc_col in ['air_temp_set_1','snow_water_equiv_set_1']:
                        if anc_col in obs.keys():
                            d[anc_col] = obs[anc_col]
                if d:
                    d['elev'] = stat['ELEVATION']
                    d['lat'] = stat['LATITUDE']
                    d['long'] = stat['LONGITUDE']
                    d['tz'] = stat['TIMEZONE']
                    res[stat['NAME']] = d
            
            stat_ls.append(stat['NAME'].lower().replace(' ',''))


larger than 0.1 diff: 4 else 34
